In [67]:
import plotly.express as px
from utils.ops import load_yaml
from  pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [88]:
site = 2
site_label  = 'PA'

In [89]:
cfg = load_yaml('cfg.yaml')
site_cfg = load_yaml(f'site_{site}.yaml')

preparation_params = cfg['preparation_params']
paths_params = cfg['paths']
general_params = cfg['general_params']
experiments_paths = general_params['experiments_folders']

In [110]:
colors = [
    'aqua',
    'chocolate',
    'darkgreen',
    'brown',
    'cornflowerblue',
    'gold',
    'lightcoral',
    'cadetblue',
    'lightgreen',
    'cyan',
    'mediumpurple',
]

In [94]:
def results(experiments):
    exp_results = []
    for exp_i in experiments:
        #experiment_params = cfg['experiments'][f'exp_{experiments[exp_i]}']
        exp_path = Path(paths_params['experiments']) / f'exp_{exp_i}'
        experiment_params = cfg['experiments'][f'exp_{exp_i}']

        results_path = exp_path / experiments_paths['results']
        final_results_file = results_path / f'results_{exp_i}.data'

        result_df = pd.read_pickle(final_results_file)
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Global',
                'metric': 'Precision',
                'value': result_df['global_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Global',
                'metric': 'Recall',
                'value': result_df['global_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Global',
                'metric': 'F1-Score',
                'value': result_df['global_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'No Cloud',
                'metric': 'Precision',
                'value': result_df['no_cloud_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'No Cloud',
                'metric': 'Recall',
                'value': result_df['no_cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'No Cloud',
                'metric': 'F1-Score',
                'value': result_df['no_cloud_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Cloudy',
                'metric': 'Precision',
                'value': result_df['cloud_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Cloudy',
                'metric': 'Recall',
                'value': result_df['cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Cloudy',
                'metric': 'F1-Score',
                'value': result_df['cloud_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        
        
    return pd.DataFrame(exp_results)


In [148]:
experiments = [
    6,
    1,
    2,
    3,
    4,
    5,
    7,
    8,
    10,
    26,
    21,
    22,
    23,
    24,
    25
]
pixel_cond = 'Global'
metric = 'F1-Score'
exp_results = results(experiments)
filt_res = exp_results[(exp_results['Cloud Condition'] == pixel_cond) & (exp_results['metric'] == metric)]
fig = make_subplots(rows=1, cols = 2, subplot_titles=('CNN', 'SWIN-Unet'), shared_yaxes = True)

cnn_results = filt_res[filt_res['group_model'] == 'CNN']
for i, name_res in enumerate(cnn_results['name']):
    loc_res = cnn_results[cnn_results['name'] == name_res]
    #print(loc_res)
    fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], text=loc_res['value'], texttemplate='%{text:.2f}', textposition='outside', row=1, col=1, marker_color = px.colors.qualitative.Plotly[i])

swin_results = filt_res[filt_res['group_model'] == 'SWIN-Unet']
for i, name_res in enumerate(swin_results['name']):
    loc_res = swin_results[swin_results['name'] == name_res]
    #print(loc_res)
    fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], showlegend = False, text=loc_res['value'], texttemplate='%{text:.2f}', textposition='outside', row=1, col=2, marker_color = px.colors.qualitative.Plotly[i])


fig.update_yaxes(range=[0.44, 1], title_text = '', tickformat = '.1f') 
fig.update_layout(
    width = 1600,
    height = 500,
    legend_title_text = 'Model', 
    legend_font_size = 16,
    #legend_orientation = 'h',
    font_size = 12,
    bargap=0.30,
    title_text = f'{metric} ({pixel_cond}) [Site:{site_label}]',
    title_font_size = 20,
    title_x = 0.5
    )
fig.show()
